In [1]:
import pandas as pd
import numpy as np
import sys
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from pandas import HDFStore
import matplotlib.pyplot as pl
import lasagne as ls
from theano import tensor as T
from lasagne.layers import InputLayer, DenseLayer
from lasagne.updates import nesterov_momentum,sgd
from lasagne.nonlinearities import rectify
from nolearn.lasagne import NeuralNet
from DemoPyEvolve import PyEvolve
from ConfigParser import SafeConfigParser
from __init__ import *
store = HDFStore("storeTraffic.h5")
#"ita_public_tools/output/data.csv"
data = pd.Series.from_csv("10min_workload.csv",header=None,index_col=None)
def read_config():
    parser = SafeConfigParser()
    parser.read('configNeural.cfg')
    hidden_layer = int(parser.get("Neural","hidden_layer"))
    epochs = int(parser.get("Neural","epochs"))
    return hidden_layer, epochs
def saveResult(nn_rmse,nn_map,nn_r2,gn_rmse,gn_map,gn_r2):
    temp = np.zeros(6,dtype=np.float64)
#     if(nn_rmse<=gn_rmse):
#         temp[0]=gn_rmse
#         temp[1]=gn_map
#         temp[2]=gn_r2
#         temp[3]=nn_rmse
#         temp[4]=nn_map
#         temp[5]=nn_r2
#     else:
    temp[0]=nn_rmse
    temp[1]=nn_map
    temp[2]=nn_r2
    temp[3]=gn_rmse
    temp[4]=gn_map
    temp[5]=gn_r2
    return temp
def mean_percentage_error(y_pred,y_actual):
    'Calculate the mean percentage absolute error'
    n = y_pred.shape[0]
    temp = [np.abs((i-j)/j) for i,j in zip(y_pred,y_actual)]
    return (1.0/n) * np.sum(temp)
class LoadParam():
    def initNN(self):
        #Build layer for MLP
        hidden_layer, epochs = read_config()
        l_in = ls.layers.InputLayer(shape=(None,self.n_input+self.n_periodic),input_var=None)
        l_hidden = ls.layers.DenseLayer(l_in,num_units=hidden_layer,nonlinearity=ls.nonlinearities.rectify)
        network = l_out = ls.layers.DenseLayer(l_hidden,num_units=1)
#         print "Neural network initialize"
        #Init Neural net
        net1 = NeuralNet(
            layers=network,
            # optimization method:
            update=nesterov_momentum,
            update_learning_rate=0.000001,
            update_momentum=0.9,
            regression=True,  # flag to indicate we're dealing with regression problem
            max_epochs=800,  # we want to train this many epochs
            eval_size = 0.4
#             verbose=1,
        )
        return net1
    def initGN(self,params=None):
        self.l_in = ls.layers.InputLayer(shape=(None,self.n_input+self.n_periodic),input_var=None,W=params.T)
        self.l_hidden = ls.layers.DenseLayer(self.l_in,num_units=15,nonlinearity=ls.nonlinearities.rectify)
        self.network = l_out = ls.layers.DenseLayer(self.l_hidden,num_units=1)
            #Init Neural net
        net1 = NeuralNet(
            layers=self.network,
            # optimization method:
            update=nesterov_momentum,
            update_learning_rate=0.000001,
            update_momentum=0.9,
            regression=True,  # flag to indicate we're dealing with regression problem
            max_epochs=800,  # we want to train this many epochs
#                 verbose=1,
            eval_size = 0.4
        )
        return net1
    def __init__(self,n_type,n_input,n_periodic=0):
        self.n_input = n_input
        self.n_periodic = n_periodic
        self.n_type = n_type
        if(n_periodic==0):
            self.net = self.initNN()
            if(n_type=="NN"):
                self.net.load_params_from('Params/saveNeuralNetwork_1e-05_%s.tdn'%n_input)
            elif(n_type=="GN"):
                self.net.load_params_from('GeneticParams/saveNeuralNetwork_1e-05_%s.tdn'%n_input)
        else:
            self.net = self.initNN()
            if(n_type=="NN"):
                self.net.load_params_from('ParamsPeriodic/saveNeuralNetwork_1e-05_%s.tdn'%n_input)
            elif(n_type=="GN"):
                self.net.load_params_from('GeneticParamsPeriodic/saveNeuralNetwork_1e-05_%s.tdn'%n_input)
    def normalize(self,dataCount,dataTest):
        dataNorm = pd.Series(np.zeros(dataCount.shape[0]),dtype=np.float64)
        dataNorm = (dataCount - dataTest.min())/(dataTest.max()-dataTest.min())
        return dataNorm
    def normalize(self,dataCount):
        dataNorm = pd.Series(np.zeros(dataCount.shape[0]),dtype=np.float64)
        dataNorm = (dataCount - dataCount.min())/(dataCount.max()-dataCount.min())
        return dataNorm
    def convert(self,data):
        max_val = float(self.workload.max())
        min_val = float(self.workload.min())
        return (data*(max_val-min_val)+min_val)
    def generate(self,range_training,range_test=1):
        # In[62]:
#         print "Loading storage"
#         print "generate data"
        self.workload = data[142*range_training[0]-self.n_input:142*range_training[1]]
        data_training = self.normalize(self.workload)
        X_training = self.getTraining(self.workload)
#         data_validation = data[142*range_training[1]-self.n_input:142*(range_training+range_test)]
        data_test = data[142*range_training[0]:142*range_training[1]]
        return np.asarray(X_training),np.asarray(data_test)
    def getTraining(self,workload):
        raw_data = data
        data_training = self.normalize(workload)
        max_val = float(workload.max())
        min_val = float(workload.min())
#         print max_val, min_val
        n_row = data_training.shape[0]
#         print "Generate X_traing, y_traing"
#         print "X_training loading..."
    #     X_training = np.asarray([[data.iloc[t-i-1] for i in range(0,n_input)]
    #                  for t in np.arange(n_input,n_row)])
        X_training = []
        for t in range(self.n_input,n_row):
            temp = []
            for i in range(0,self.n_input):
#                 print data_training.iloc[t-i-1]
                temp.append(data_training.iloc[t-i-1])
            for j in range(1,self.n_periodic+1):
                start_idx = data_training.index[t]
                norVal = (raw_data[start_idx-142*j]-min_val)/(max_val-min_val)
#                 print raw_data[start_idx-142*j]
                temp.append(norVal)
            X_training.append(temp)
        return X_training
    def predict(self,X_test):
#         dataTest= pd.read_sql("SELECT count FROM workload where time >= 895096802-%d and time < 895096802+86400"%(n_input),conn)["count"]
        return self.net.predict(X_test)
    def score(self,X_test,y_actual):
        return self.net.score(X_test,y_actual)
#     def plot_loss(self):
#         """
#         Plot the training loss and validation loss versus epoch iterations with respect to 
#         a trained neural network.
#         """
#         net = self.net
#         train_loss = np.array([i["train_loss"] for i in net.train_history_])
#         valid_loss = np.array([i["valid_loss"] for i in net.train_history_])
#         pl.plot(train_loss, linewidth = 3, label = "train")
#         pl.plot(valid_loss, linewidth = 3, label = "valid")
#         pl.grid()
#         pl.legend()
#         pl.xlabel("epoch")
#         pl.ylabel("loss")
#         #pyplot.ylim(1e-3, 1e-2)
#         pl.yscale("log")
#         pl.show()
    def plot_loss(self,train_loss,valid_loss):
        """
        Plot the training loss and validation loss versus epoch iterations with respect to 
        a trained neural network.
        """
        pl.plot(train_loss, linewidth = 2, label = "train")
        pl.plot(valid_loss, linewidth = 2, label = "valid")

        pl.legend()
        pl.xlabel("epoch")
        pl.ylabel("loss")
        #pyplot.ylim(1e-3, 1e-2)
#         pl.yscale("log")
        pl.show()
    def fitTraining(self,X_training,y_training):
        if(self.n_type=="GN"):
            geneticEngine = PyEvolve(n_input)
            geneticEngine.fit()
            nnParams = geneticEngine.getParam()
            self.net = self.initGN(nnParams)
        else:
            self.net = self.initNN()
        self.net.fit(X_training,y_training)

/home/nhuan/anaconda2/lib/python2.7/site-packages/Theano-0.8.0.dev0-py2.7.egg/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [2]:
list_nresult = []
for n_input in np.arange(2,21):
    temp = np.zeros(6)
    temp=[]
#     n_input = 12
    nn = LoadParam("GN",n_input)
    gn = LoadParam("GN",n_input,1)
    #     print "With input"
    #     for i in np.arange(1,data[0:142*30].shape[0],1):
    i = 46
    skip_list = 2
    #     print "%d-%d"%(i,i+skip_list)
    # X_training,y_training = nn.generate((i,i+skip_list))
    X_test,y_test = nn.generate((i,i+skip_list))
    X_ptest,y_ptest = gn.generate((i,i+skip_list))
        # Xp_training,yp_training = nnp.generate((i,i+skip_list))
        # Xp_test,yp_test = nnp.generate((i+skip_list+1,i+skip_list+2))
        # nn.fitTraining(X_training,y_training)
        # nnp.fitTraining(Xp_training,yp_training)
        # dataX = data[142*3:142*5]
    #     print "NN score = %f"%nn.score(X_test,y_test)
    #     print "GN score = %f"%gn.score(X_ptest,y_ptest)
    #     temp.append(nn.score(nn.convert(X_test),nn.convert(y_test)))
    nn_pred = nn.convert(nn.predict(X_test))
    gn_pred = gn.convert(gn.predict(X_ptest))
    
    temp.append(np.sqrt(mean_squared_error(nn_pred,y_test)))
    temp.append(mean_absolute_error(nn_pred,y_test))
    temp.append(r2_score(y_test,nn_pred))
    temp.append(mean_percentage_error(y_test,nn_pred))
    
    temp.append(np.sqrt(mean_squared_error(gn_pred,y_ptest)))
    temp.append(mean_absolute_error(gn_pred,y_ptest))
    temp.append(r2_score(y_ptest,gn_pred))
    temp.append(mean_percentage_error(y_ptest,gn_pred))
    
    list_nresult.append(temp)

/home/nhuan/anaconda2/lib/python2.7/site-packages/nolearn-0.6a0.dev0-py2.7.egg/nolearn/lasagne/base.py:250: UserWarning: The 'eval_size' argument has been deprecated, please use the 'train_split' parameter instead, e.g.
train_split=TrainSplit(eval_size=0.4)
  warn("The 'eval_size' argument has been deprecated, please use "


AttributeError: 'module' object has no attribute 'Param'

In [4]:
import theano
theano.__version__

'0.8.0.dev0.dev-e891fb3fed4c1522f23a201bda6cfee6cf115e8b'

In [28]:
n_input = 4
nn = LoadParam("GN",4)
gn = LoadParam("GN",n_input,1)
i = 46
skip_list = 4
X_test,y_test = nn.generate((i,i+skip_list))
X_ptest,y_ptest = gn.generate((i,i+skip_list))

gn_pred = gn.convert(gn.predict(X_ptest))
nn_pred = nn.convert(nn.predict(X_test))
y_actual = y_test
ax2 = pl.subplot()
ax2.set_color_cycle(['blue','red','green'])
ax2.plot(nn_pred,'--',label="Without Periodic Input")
ax2.plot(gn_pred,'--',label="With Periodic Input")
ax2.plot(y_actual,label="Actual")
ax2.set_title("Genetic Neural Network with Periodic Input based Prediction (Sliding Window Size = 4)")
ax2.set_ylabel("Connections")
ax2.set_xlabel("Time (minutes)")
ax2.set_color_cycle(['blue','red'])
ax2.legend()
pl.show()

In [3]:
a = NeuralNet()

TypeError: __init__() takes at least 2 arguments (1 given)

In [11]:
list_nresult= pd.DataFrame(list_nresult,columns=["RMSE_NN","MAE_NN","R2_NN","MAPE_NN","RMSE_GN","MAE_GN","R2_GN","MAPE_GN"],index=np.arange(2,21))
list_nresult

,RMSE_NN,MAE_NN,R2_NN,MAPE_NN,RMSE_GN,MAE_GN,R2_GN,MAPE_GN
2,328552.299152,210383.974363,-0.695429,1.630024,70656.170502,33742.822162,0.921590,0.071340
3,124019.990982,79489.201237,0.758424,0.225141,71304.611556,36076.375865,0.920144,0.077698
4,104314.704910,65649.546090,0.829092,0.174545,51738.555057,26915.557516,0.957956,0.061221
5,217303.288100,148541.217068,0.258343,0.477888,328554.336546,210425.172535,-0.695450,1.630381
6,297306.045762,175616.015200,-0.388283,0.984764,60548.612695,31186.695624,0.942419,0.066647
7,123904.541936,76604.986607,0.758873,0.201801,65850.114461,32412.832290,0.931894,0.067321
8,192499.365168,136252.717521,0.417992,0.454460,59535.709836,29425.746252,0.944329,0.063149
9,201280.991082,133064.109023,0.363679,0.423395,69010.809400,36011.018284,0.925199,0.073061
10,175159.795779,112001.538198,0.518119,0.336259,57834.961503,30172.344352,0.947465,0.067356
11,285179.942668,175536.882830,-0.277346,0.884263,73754.053112,38309.676224,0.914564,0.084385


In [ ]:
%matplotlib
list_nresult["MAPE_GN"].plot(kind="line")
list_nresult["MAPE_NN"].plot(kind="line")

In [ ]:
result_score = pd.DataFrame(list_nresult,columns=["RMSE_GNP","MAE_GNP","R2_GNP"],index=np.arange(4,16))
result_score
# temp = result_score["MAE_GN"][13]
# result_score["MAE_GN"][13]=result_score["MAE_GN"][11]
# result_score["MAE_GN"][11]=temp

In [18]:
%matplotlib
ax1 = list_nresult["MAPE_GN"].plot(kind='line',label='MAPE GNP',color='red',title="MAE vs. Window Size (Genetic Neural Network with Periodic)",grid=False)
ax2 = list_nresult["MAPE_NN"].plot(kind='line',label='MAPE GN', color='blue',title="MAE vs. Window Size (Genetic Neural Network with Periodic)",grid=False)
ax1.set_xlabel("Sliding Window size")
ax1.set_ylabel("Mean Absolute Error (MAE)")
ax1.legend()

Using matplotlib backend: TkAgg


In [19]:
store = HDFStore("experiement")

In [23]:
store["mape_gnp"] = list_nresult

In [75]:
np.savez("experiement_12-1",gn_pred=gn_pred,nn_pred = nn_pred,mape_gnp=list_nresult,y_actual=y_actual)